In [ ]:
import numpy as np
import pandas as pd
import optuna
from scipy.sparse import csr_matrix
from weighting_strategies import (
    bm25_weight, tfidf_weight, normalized_weight,
    log_weight, confidence_weight, power_weight,
    pmi_weight, robust_user_centric_weight, sigmoid_propensity_weight, power_lift_weight, robust_user_centric_weight_v2
)
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import train_test_split, precision_at_k, ndcg_at_k

import cornac


In [2]:
import sys
import os

# Add the parent directory to sys.path to resolve imports from sibling directories
sys.path.append(os.path.abspath(".."))

from utils.sparse import transform_dataframe_to_sparse

In [3]:
movielens_df = (
    pd.DataFrame(
        data=cornac.datasets.movielens.load_feedback(variant="100K"),
        columns=['user_id', 'item_id', 'target']
    )
    .loc[:, ['user_id', 'item_id', 'target']]
    .dropna()
)
movielens_df['user_id'].nunique(), movielens_df['item_id'].nunique(), movielens_df.shape[0]

(943, 1682, 100000)

In [4]:
user_item_matrix, user_mapping, item_mapping = transform_dataframe_to_sparse(
    movielens_df, row_field='user_id', col_field='item_id', data_field='target'
)


train_val_mat, test_mat = train_test_split(user_item_matrix, train_percentage=0.9, random_state=42)
train_mat, val_mat = train_test_split(train_val_mat, train_percentage=0.9, random_state=42)

print(f"Train Shape: {train_mat.shape}, Val Shape: {val_mat.shape}, Test Shape: {test_mat.shape}")

Train Shape: (943, 1682), Val Shape: (943, 1682), Test Shape: (943, 1682)


In [ ]:
results_folder = "results/movielens_100k"
results_filename = "movielens_100k_experiment_results.csv"

import time

def run_hyperparameter_optimization(
    train_mat: csr_matrix,
    val_mat: csr_matrix,
    train_val_mat: csr_matrix,
    test_mat: csr_matrix,
    weighting_strategy: str,
    algorithm: str,
    n_trials: int = 20,
    output_dir: str = None,
) -> pd.DataFrame:
    results = []
    algorithms = {
        "ALS_factors=10": lambda: AlternatingLeastSquares(factors=10, random_state=42),
        "ALS_factors=100": lambda: AlternatingLeastSquares(factors=100, random_state=42),
    }
    strategies = [
        "no_weighting",
        "bm25",
        "tfidf",
        "log", 
        "confidence",
        "power",
        "normalized",
        "pmi",
        "robust_user_centric",
        "robust_user_centric_weight_v2",
        "sigmoid_propensity",
        "power_lift"
    ]
    if weighting_strategy not in strategies:
        raise ValueError(f"Weighting strategy '{weighting_strategy}' is not recognized.")
    strategy = weighting_strategy

    if algorithm not in algorithms:
        raise ValueError(f"Algorithm '{algorithm}' is not recognized.")
    algo_name = algorithm
    AlgoFactory = algorithms[algorithm]

    print(f"Running optimization for {algo_name} with {strategy}...")

    def get_weighted_matrix(matrix, params):
        weighted = matrix.copy()
        if strategy == "bm25":
            weighted = bm25_weight(weighted, K1=params.get("bm25_k1"), B=params.get("bm25_b"))
        elif strategy == "confidence":
            weighted = confidence_weight(weighted, alpha=params.get("conf_alpha"))
        elif strategy == "power":
            weighted = power_weight(weighted, p=params.get("power_p"))
        elif strategy == "tfidf":
            weighted = tfidf_weight(weighted)
        elif strategy == "log":
            weighted = log_weight(weighted)
        elif strategy == "normalized":
            weighted = normalized_weight(weighted)
        elif strategy == "pmi":
            weighted = pmi_weight(weighted)
        elif strategy == "robust_user_centric":
            weighted = robust_user_centric_weight(weighted, scale_factor=params.get("scale_factor"))
        elif strategy == "sigmoid_propensity":
            weighted = sigmoid_propensity_weight(weighted, p=params.get("p"), beta=params.get("beta"))
        elif strategy == "power_lift":
            weighted = power_lift_weight(weighted, p=params.get("p"))
        elif strategy == "robust_user_centric_weight_v2":
            weighted = robust_user_centric_weight_v2(weighted, lower_q=params.get("lower_q"), upper_q=params.get("upper_q"))
        return weighted

    def objective(trial):
        params = {}
        # Suggest weighting strategy parameters
        if strategy == "bm25":
            params["bm25_k1"] = trial.suggest_float("bm25_k1", 0.1, 1000)
            params["bm25_b"] = trial.suggest_float("bm25_b", 0.0, 1.0)
        elif strategy == "confidence":
            params["conf_alpha"] = trial.suggest_float("conf_alpha", 1.0, 150.0)
        elif strategy == "power":
            params["power_p"] = trial.suggest_float("power_p", 0.1, 1.5)
        elif strategy == "robust_user_centric":
            params["scale_factor"] = trial.suggest_float("scale_factor", 0.1, 10.0)
        elif strategy == "robust_user_centric_weight_v2":
            params["lower_q"] = trial.suggest_float("lower_q", 5.0, 45.0)
            params["upper_q"] = trial.suggest_float("upper_q", 55.0, 95.0)
        elif strategy == "sigmoid_propensity":
            params["p"] = trial.suggest_float("p", 0.1, 5.0)
            params["beta"] = trial.suggest_float("beta", 0.0, 1.0)
        elif strategy == "power_lift":
            params["p"] = trial.suggest_float("p", 0.1, 1.5)
        weighted_train = get_weighted_matrix(train_mat, params)
        
        # Train Model
        model = AlgoFactory()
        model.fit(weighted_train, show_progress=False)

        # Evaluate on Validation Set
        return ndcg_at_k(model, train_mat, val_mat, K=20, show_progress=False)

    # Optimize only if strategy has parameters
    current_trials = n_trials if strategy in ["bm25", "confidence", "power", "robust_user_centric", "robust_user_centric_weight_v2", "sigmoid_propensity", "power_lift"] else 1
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=current_trials, n_jobs=-1)

    # --- Final Retraining & Testing ---
    # Use best params to weight the full train_val matrix
    best_params = study.best_params
    weighted_train_val = get_weighted_matrix(train_val_mat, best_params)

    # Train Final Model
    final_model = AlgoFactory()
    
    start_time = time.time()
    final_model.fit(weighted_train_val, show_progress=False)
    end_time = time.time()
    
    # Evaluate on Test Set
    test_ndcg_10 = ndcg_at_k(final_model, train_val_mat, test_mat, K=10, show_progress=False)
    test_precision_10 = precision_at_k(final_model, train_val_mat, test_mat, K=10, show_progress=False)
    test_ndcg_20 = ndcg_at_k(final_model, train_val_mat, test_mat, K=20, show_progress=False)
    test_precision_20 = precision_at_k(final_model, train_val_mat, test_mat, K=20, show_progress=False)

    results.append({
        "Algorithm": algo_name,
        "Strategy": strategy,
        "Number of Optimization Trials": current_trials,
        "Best Val NDCG@20": study.best_value,
        "Test NDCG@10": test_ndcg_10,
        "Test NDCG@20": test_ndcg_20,
        "Test Precision@10": test_precision_10,
        "Test Precision@20": test_precision_20,
        "Final Train Time (s)": end_time - start_time,
        "Best Params": best_params
    })

    if output_dir:
        output_path = os.path.join(output_dir, f"{algo_name}_{strategy}_results.csv")
        pd.DataFrame(results).to_csv(output_path, index=False)
    return pd.DataFrame(results)

In [ ]:
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

# Execution of all experiments
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="no_weighting", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="no_weighting", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="bm25", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="bm25", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="tfidf", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="tfidf", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="log", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="log", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="confidence", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="confidence", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="normalized", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="normalized", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="pmi", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="pmi", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric_weight_v2", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric_weight_v2", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="sigmoid_propensity", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="sigmoid_propensity", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power_lift", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power_lift", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-01-29 21:35:49,832] A new study created in memory with name: no-name-768b3392-56c4-480e-897d-8c5116b7b2c9


Running optimization for ALS_factors=10 with no_weighting...


[I 2026-01-29 21:35:50,087] Trial 0 finished with value: 0.27244925287930244 and parameters: {}. Best is trial 0 with value: 0.27244925287930244.
[I 2026-01-29 21:35:50,128] A new study created in memory with name: no-name-f5db5fc9-e47a-4c01-a08c-57c7816623e8
[I 2026-01-29 21:35:50,249] Trial 0 finished with value: 0.2550847356281346 and parameters: {'bm25_k1': 392.9630961200425, 'bm25_b': 0.20702281961533286}. Best is trial 0 with value: 0.2550847356281346.
[I 2026-01-29 21:35:50,257] Trial 1 finished with value: 0.25914799539540434 and parameters: {'bm25_k1': 18.49902926772266, 'bm25_b': 0.7483218884291787}. Best is trial 1 with value: 0.25914799539540434.


Running optimization for ALS_factors=10 with bm25...


[I 2026-01-29 21:35:50,426] Trial 3 finished with value: 0.25441476754844666 and parameters: {'bm25_k1': 997.9611788813592, 'bm25_b': 0.6954266746749062}. Best is trial 1 with value: 0.25914799539540434.
[I 2026-01-29 21:35:50,441] Trial 2 finished with value: 0.2551876705787217 and parameters: {'bm25_k1': 382.11347123530504, 'bm25_b': 0.5964279163182016}. Best is trial 1 with value: 0.25914799539540434.
[I 2026-01-29 21:35:50,608] Trial 5 finished with value: 0.25478032825910524 and parameters: {'bm25_k1': 360.1119500875812, 'bm25_b': 0.48011264799868814}. Best is trial 1 with value: 0.25914799539540434.
[I 2026-01-29 21:35:50,633] Trial 4 finished with value: 0.254138725201777 and parameters: {'bm25_k1': 348.91783013800216, 'bm25_b': 0.2657594477163353}. Best is trial 1 with value: 0.25914799539540434.
[I 2026-01-29 21:35:50,727] Trial 6 finished with value: 0.25069574461272587 and parameters: {'bm25_k1': 687.0021561733502, 'bm25_b': 0.03312977932847816}. Best is trial 1 with value: 

...

Running optimization for ALS_factors=100 with sigmoid_propensity...


[I 2026-01-29 21:36:34,045] Trial 3 finished with value: 0.21978194131601553 and parameters: {'p': 1.267129304001736, 'beta': 0.9081640847987025}. Best is trial 3 with value: 0.21978194131601553.
[I 2026-01-29 21:36:34,062] Trial 1 finished with value: 0.21629484572171764 and parameters: {'p': 2.337177107420177, 'beta': 0.5505157521025191}. Best is trial 3 with value: 0.21978194131601553.
[I 2026-01-29 21:36:34,071] Trial 0 finished with value: 0.2184135609869795 and parameters: {'p': 1.4472188847245229, 'beta': 0.7282328023555357}. Best is trial 3 with value: 0.21978194131601553.
[I 2026-01-29 21:36:34,072] Trial 2 finished with value: 0.19951102407902513 and parameters: {'p': 1.754583261209572, 'beta': 0.05339150693109784}. Best is trial 3 with value: 0.21978194131601553.
[I 2026-01-29 21:36:34,326] Trial 4 finished with value: 0.2085639456465716 and parameters: {'p': 4.498423599870919, 'beta': 0.5092912850902908}. Best is trial 3 with value: 0.21978194131601553.
[I 2026-01-29 21:36:

Running optimization for ALS_factors=100 with power_lift...


[I 2026-01-29 21:36:35,905] Trial 0 finished with value: 0.19491544361222762 and parameters: {'p': 1.0028780566704256}. Best is trial 0 with value: 0.19491544361222762.
[I 2026-01-29 21:36:36,013] Trial 1 finished with value: 0.2124158633050557 and parameters: {'p': 0.253216302866053}. Best is trial 1 with value: 0.2124158633050557.
[I 2026-01-29 21:36:36,121] Trial 2 finished with value: 0.20738866968265482 and parameters: {'p': 0.4657381146829297}. Best is trial 1 with value: 0.2124158633050557.
[I 2026-01-29 21:36:36,131] Trial 3 finished with value: 0.19555704400855548 and parameters: {'p': 1.070329076645179}. Best is trial 1 with value: 0.2124158633050557.
[I 2026-01-29 21:36:36,375] Trial 5 finished with value: 0.18499684180119594 and parameters: {'p': 1.451454831506804}. Best is trial 1 with value: 0.2124158633050557.
[I 2026-01-29 21:36:36,478] Trial 4 finished with value: 0.19525502772577927 and parameters: {'p': 1.159018357297566}. Best is trial 1 with value: 0.21241586330505

,Algorithm,Strategy,Best Val NDCG@20,Test NDCG@20,Test Precision@20,Final Train Time (s),Best Params
11,ALS_factors=10,power_lift,0.275097,0.339081,0.373401,0.029398,{'p': 0.20298154336881027}
7,ALS_factors=10,pmi,0.265106,0.332565,0.358479,0.030410,{}
5,ALS_factors=10,power,0.276861,0.331789,0.372335,0.030452,{'power_p': 0.5504693184151779}
3,ALS_factors=10,log,0.274856,0.330926,0.370914,0.032584,{}
10,ALS_factors=10,sigmoid_propensity,0.273607,0.330089,0.370914,0.030450,"{'p': 1.40152528826798, 'beta': 0.568393012901..."
0,ALS_factors=10,no_weighting,0.272449,0.329265,0.367480,0.030377,{}
8,ALS_factors=10,robust_user_centric,0.272076,0.327652,0.367598,0.030441,{'scale_factor': 2.8983133438111786}
1,ALS_factors=10,bm25,0.260228,0.324415,0.358835,0.034649,"{'bm25_k1': 10.924518246974912, 'bm25_b': 0.47..."
2,ALS_factors=10,tfidf,0.258425,0.322167,0.356111,0.033605,{}
4,ALS_factors=10,confidence,0.260652,0.321578,0.355045,0.029398,{'conf_alpha': 1.1958441973755038}


In [ ]:
import glob

all_results = []
# Match any CSV in the result folder
for f in glob.glob(f"{results_folder}/*.csv"):
    all_results.append(pd.read_csv(f))

if all_results:
    experiment_results = pd.concat(all_results)
    experiment_results = experiment_results.sort_values("Test NDCG@20", ascending=False)
    experiment_results.to_csv(results_filename, index=False)
else:
    print("No results found.")

experiment_results